In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from datetime import datetime, timedelta
%matplotlib inline
import yfinance as yf
import mibian as mb

In [2]:
msft_df = pd.read_pickle('msft_merged.pickle')

In [3]:
df = msft_df
df = df.drop(columns= ['LastPrice', 'Symbol', 'Volume', 'OpenInterest', 'Volume'])
#df['OptionKey'] =df['OptionKey'].map(lambda x: str(x)[:-10])


In [4]:
df['ExpirationDate'] =  pd.to_datetime(df['ExpirationDate'], infer_datetime_format=True)
df['DataDate'] =  pd.to_datetime(df['DataDate'], infer_datetime_format=True)
df['delta'] = df['ExpirationDate'] - df['DataDate']

df['exp?'] =df['ExpirationDate'] ==df['DataDate']
df['long_close'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.1) & (df['% out'] < 0))
df['long_medium'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.2) & (df['% out'] < -.1))
df['long_far'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.3) & (df['% out'] < -.2))
df['short_close'] = ((df['delta'] < np.timedelta64(8, 'D')) & (df['% out'] > -.1) & (df['% out'] < 0))
df['short_medium'] = ((df['delta'] < np.timedelta64(8, 'D')) & (df['% out'] > -.2) & (df['% out'] < -.1))
df['short_far'] = ((df['delta'] < np.timedelta64(8, 'D')) & (df['% out'] > -.3) & (df['% out'] < -.2))

#df['collatarel'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.4) & (df['% out'] < -.2))
df['Signal'] = np.random.randint(-1,2, df.shape[0])
#df.loc[(df['Signal'] == 1) & (df['worth_it'] == True), 'execute']=df.loc[(df['Signal'] == 1) & (df['worth_it'] == True), 'AskPrice']
#df['enter?'] = (df['worth_it'] == True) & (df['Signal'] == 1)
#df.loc[df['enter?'] == True]
pd.set_option('display.max_columns', 30)

In [5]:
df_table=df.loc[:, ['OptionKey', 'BidPrice']]
df_table=df_table.reset_index()
df_table.columns=['PriceDate', 'OptionKey', 'RebuyPrice']
df_table['RebuyPrice'].nunique()

416

In [6]:
#msft_df_table['PriceDate'] =  pd.to_datetime(msft_df_table['PriceDate'], infer_datetime_format=True)
#msft_result = pd.merge(msft_df, msft_df_table, left_on=['ExpirationDate', 'OptionKey'], right_on=['PriceDate', 'OptionKey'], how='left')
#msft_result

In [7]:
#ok=msft_result.columns.values.tolist()
#for column_name in ok:
 #  print(msft_result['RebuyPrice'].unique())

In [8]:
#exp = (df.loc[df['exp?'] == True], 

In [9]:
xp = df
xp.loc[xp['exp?']== True, 'xp' ] = xp.loc[df['exp?'] == True, 'AskPrice']
xp = xp.loc[xp['exp?']== True]
xp= xp[['OptionKey', 'xp']]
xp

,OptionKey,xp
2019-11-15,MSFT2019-11-15P125.00,0.01
2019-11-15,MSFT2019-11-15P127.00,0.01
2019-11-15,MSFT2019-11-15P129.00,0.45
2019-11-15,MSFT2019-11-15P130.00,0.01
2019-11-15,MSFT2019-11-15P131.00,0.03
...,...,...
2019-12-13,MSFT2019-12-13P150.00,0.01
2019-12-13,MSFT2019-12-13P152.50,0.01
2019-12-13,MSFT2019-12-13P155.00,0.68
2019-12-13,MSFT2019-12-13P157.50,3.85


In [10]:
#mf = exp
#mf = mf[['OptionKey', 'xp']]
#mf = mf.dropna()
#mf
#msft_xp

In [11]:
df['xp']=df.OptionKey.map(xp.set_index('OptionKey')['xp'])
df

,OptionKey,ExpirationDate,AskPrice,BidPrice,PutCall,StrikePrice,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value,LSTM OTM Cr Entry,RF OTM Cr Entry,Bid Price,delta,exp?,long_close,long_medium,long_far,short_close,short_medium,short_far,Signal,xp
2019-11-13,MSFT2019-11-15P120.00,2019-11-15,0.01,0.00,put,120.0,147.31,2019-11-13,2 days,-0.185391,1,1,0.0,0.0,0.0,2 days,False,False,True,False,False,True,False,1,NaN
2019-11-13,MSFT2019-11-15P125.00,2019-11-15,0.01,0.00,put,125.0,147.31,2019-11-13,2 days,-0.151449,1,1,0.0,0.0,0.0,2 days,False,False,True,False,False,True,False,-1,0.01
2019-11-14,MSFT2019-11-15P125.00,2019-11-15,0.01,0.00,put,125.0,148.06,2019-11-14,1 days,-0.155748,1,1,0.0,0.0,0.0,1 days,False,False,True,False,False,True,False,-1,0.01
2019-11-15,MSFT2019-11-15P125.00,2019-11-15,0.01,0.00,put,125.0,149.97,2019-11-15,0 days,-0.166500,1,1,0.0,0.0,0.0,0 days,True,False,True,False,False,True,False,-1,0.01
2019-11-14,MSFT2019-11-15P127.00,2019-11-15,0.01,0.00,put,127.0,148.06,2019-11-14,1 days,-0.142240,1,1,0.0,0.0,0.0,1 days,False,False,True,False,False,True,False,0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-02,MSFT2019-12-27P162.50,2019-12-27,13.60,12.30,put,162.5,149.55,2019-12-02,25 days,0.086593,-1,1,0.0,1360.0,1230.0,25 days,False,False,False,False,False,False,False,0,NaN
2019-12-13,MSFT2019-12-27P162.50,2019-12-27,10.15,6.45,put,162.5,154.53,2019-12-13,14 days,0.051576,1,1,1015.0,1015.0,645.0,14 days,False,False,False,False,False,False,False,1,NaN
2019-11-19,MSFT2019-12-27P170.00,2019-12-27,21.70,18.15,put,170.0,150.39,2019-11-19,38 days,0.130394,-1,1,0.0,2170.0,1815.0,38 days,False,False,False,False,False,False,False,0,NaN
2019-11-20,MSFT2019-12-27P170.00,2019-12-27,22.25,19.45,put,170.0,149.62,2019-11-20,37 days,0.136212,-1,1,0.0,2225.0,1945.0,37 days,False,False,False,False,False,False,False,-1,NaN


In [12]:
short_close = df.loc[(df['short_close']== True) & (df['Signal'] == 1)]
short_close['log'] = short_close['AskPrice'] - short_close['xp']
short_close_total = short_close['log'].sum()
print(short_close_total)

14.170000000000002


<ipython-input-12-db0ecfdb9cad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_close['log'] = short_close['AskPrice'] - short_close['xp']


In [13]:
short_close_RF = df.loc[(df['short_close']== True) & (df['RF Predicted Value'] == 1)]
short_close_RF['log'] = short_close_RF['AskPrice'] - short_close_RF['xp']
short_close_RF_total = short_close_RF['log'].sum()
print(short_close_RF_total)

26.919999999999998


<ipython-input-13-26e370e263bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_close_RF['log'] = short_close_RF['AskPrice'] - short_close_RF['xp']


In [14]:
short_close_LTSM = df.loc[(df['short_close']== True) & (df['LSTM Predicted Signal'] == 1)]
short_close_LTSM['log'] = short_close_LTSM['AskPrice'] - short_close_LTSM['xp']
short_close_LTSM_total = short_close_LTSM['log'].sum()
print(short_close_LTSM_total)

8.080000000000002


<ipython-input-14-859958c1296c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_close_LTSM['log'] = short_close_LTSM['AskPrice'] - short_close_LTSM['xp']


In [15]:
short_medium = df.loc[(df['short_medium']== True) & (df['Signal'] == 1)]
short_medium['log'] = short_medium['AskPrice'] - short_medium['xp']
short_medium_total = short_medium['log'].sum()
print(short_medium_total)

0.08000000000000002


<ipython-input-15-e8b5d44531af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_medium['log'] = short_medium['AskPrice'] - short_medium['xp']


In [16]:
short_medium_RF = df.loc[(df['short_medium']== True) & (df['RF Predicted Value'] == 1)]
short_medium_RF['log'] = short_medium_RF['AskPrice'] - short_medium_RF['xp']
short_medium_RF_total = short_medium_RF['log'].sum()
print(short_medium_RF_total)

0.010000000000000009


<ipython-input-16-3e62de6047f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_medium_RF['log'] = short_medium_RF['AskPrice'] - short_medium_RF['xp']


In [17]:
short_medium_LTSM = df.loc[(df['short_medium']== True) & (df['LSTM Predicted Signal'] == 1)]
short_medium_LTSM['log'] = short_medium_LTSM['AskPrice'] - short_medium_LTSM['xp']
short_medium_LTSM_total = short_medium_LTSM['log'].sum()
print(short_medium_LTSM_total)

-0.019999999999999907


<ipython-input-17-fcf0944296be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_medium_LTSM['log'] = short_medium_LTSM['AskPrice'] - short_medium_LTSM['xp']


In [18]:
short_far = df.loc[(df['short_far']== True) & (df['Signal'] == 1)]
short_far['log'] = short_far['AskPrice'] - short_far['xp']
short_far_total = short_far['log'].sum()
print(short_far_total)

0.0


<ipython-input-18-ac8845e429a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_far['log'] = short_far['AskPrice'] - short_far['xp']


In [19]:
short_far_LTSM = df.loc[(df['short_far']== True) & (df['LSTM Predicted Signal'] == 1)]
short_far_LTSM['log'] = short_far_LTSM['AskPrice'] - short_far_LTSM['xp']
short_far_LTSM_total = short_far_LTSM['log'].sum()
print(short_far_LTSM_total)

0.0


<ipython-input-19-b3a1e7696dea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_far_LTSM['log'] = short_far_LTSM['AskPrice'] - short_far_LTSM['xp']


In [20]:
short_far_RF = df.loc[(df['short_far']== True) & (df['RF Predicted Value'] == 1)]
short_far_RF['log'] = short_far_RF['AskPrice'] - short_far_RF['xp']
short_far_RF_total = short_far_RF['log'].sum()
print(short_far_RF_total)

0.0


<ipython-input-20-18c5042b77ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_far_RF['log'] = short_far_RF['AskPrice'] - short_far_RF['xp']


In [21]:
long_close = df.loc[(df['long_close']== True) & (df['Signal'] == 1)]
long_close['log'] = long_close['AskPrice'] - long_close['xp']
long_close_total = long_close['log'].sum()
print(long_close_total)

35.39


<ipython-input-21-75a269248bd1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_close['log'] = long_close['AskPrice'] - long_close['xp']


In [22]:
long_close_LTSM = df.loc[(df['long_close']== True) & (df['LSTM Predicted Signal'] == 1)]
long_close_LTSM['log'] = long_close_LTSM['AskPrice'] - long_close_LTSM['xp']
long_close_LTSM_total = long_close_LTSM['log'].sum()
print(long_close_LTSM_total)

17.96


<ipython-input-22-3a82462531d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_close_LTSM['log'] = long_close_LTSM['AskPrice'] - long_close_LTSM['xp']


In [23]:
long_close_RF = df.loc[(df['long_close']== True) & (df['RF Predicted Value'] == 1)]
long_close_RF['log'] = long_close_RF['AskPrice'] - long_close_RF['xp']
long_close_RF_total = long_close_RF['log'].sum()
print(long_close_RF_total)

72.93


<ipython-input-23-2c96faa3b979>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_close_RF['log'] = long_close_RF['AskPrice'] - long_close_RF['xp']


In [24]:
long_far = df.loc[(df['long_far']== True) & (df['Signal'] == 1)]
long_far['log'] = long_far['AskPrice'] - long_far['xp']
long_far_total = long_far['log'].sum()
print(long_far_total)

0.0


<ipython-input-24-f50bec28ebd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_far['log'] = long_far['AskPrice'] - long_far['xp']


In [25]:
long_far_RF = df.loc[(df['long_far']== True) & (df['RF Predicted Value'] == 1)]
long_far_RF['log'] = long_far_RF['AskPrice'] - long_far_RF['xp']
long_far_RF_total = long_far_RF['log'].sum()
print(long_far_RF_total)

0.0


<ipython-input-25-42ba9debbf05>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_far_RF['log'] = long_far_RF['AskPrice'] - long_far_RF['xp']


In [26]:
long_far_LTSM = df.loc[(df['long_far']== True) & (df['LSTM Predicted Signal'] == 1)]
long_far_LTSM['log'] = long_far_LTSM['AskPrice'] - long_far_LTSM['xp']
long_far_LTSM_total = long_far_LTSM['log'].sum()
print(long_far_LTSM_total)

0.0


<ipython-input-26-7db88ed9f97f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_far_LTSM['log'] = long_far_LTSM['AskPrice'] - long_far_LTSM['xp']


In [27]:
long_medium_LTSM = df.loc[(df['long_medium']== True) & (df['LSTM Predicted Signal'] == 1)]
long_medium_LTSM['log'] = long_medium_LTSM['AskPrice'] - long_medium_LTSM['xp']
long_medium_LTSM_total = long_medium_LTSM['log'].sum()
print(long_medium_LTSM_total)

0.24000000000000007


<ipython-input-27-9fa1fb1575a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_medium_LTSM['log'] = long_medium_LTSM['AskPrice'] - long_medium_LTSM['xp']


In [28]:
long_medium_RF = df.loc[(df['long_medium']== True) & (df['RF Predicted Value'] == 1)]
long_medium_RF['log'] = long_medium_RF['AskPrice'] - long_medium_RF['xp']
long_medium_RF_total = long_medium_RF['log'].sum()
print(long_medium_RF_total)

0.4900000000000001


<ipython-input-28-29d39dcc6c84>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_medium_RF['log'] = long_medium_RF['AskPrice'] - long_medium_RF['xp']


In [29]:
long_medium = df.loc[(df['long_medium']== True) & (df['Signal'] == 1)]
long_medium['log'] = long_medium['AskPrice'] - long_medium['xp']
long_medium_total = long_medium['log'].sum()
print(long_medium_total)

0.6200000000000001


<ipython-input-29-466d7a72f370>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_medium['log'] = long_medium['AskPrice'] - long_medium['xp']


In [30]:
short_close_negatives =(short_close.loc[(short_close.log < 0)])
short_close_negatives.log.mean()
# there are 14 times we were wrong

-0.05000000000000001

In [31]:
short_close_positives =(short_close.loc[(short_close.log > 0)])
#short_close_positives.log.mean()
short_close_positives = short_close_positives.sort_values('log', ascending= False)
short_close_positives.head()

,OptionKey,ExpirationDate,AskPrice,BidPrice,PutCall,StrikePrice,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value,LSTM OTM Cr Entry,RF OTM Cr Entry,Bid Price,delta,exp?,long_close,long_medium,long_far,short_close,short_medium,short_far,Signal,xp,log
2019-12-06,MSFT2019-12-13P141.00,2019-12-13,1.07,0.05,put,141.0,151.75,2019-12-06,7 days,-0.070840,-1,0,0.0,0.0,5.0,7 days,False,True,False,False,True,False,False,1,0.01,1.06
2019-11-15,MSFT2019-11-22P149.00,2019-11-22,1.02,0.86,put,149.0,149.97,2019-11-15,7 days,-0.006468,1,1,0.0,0.0,86.0,7 days,False,True,False,False,True,False,False,1,0.03,0.99
2019-11-22,MSFT2019-11-29P149.00,2019-11-29,0.73,0.66,put,149.0,149.59,2019-11-22,7 days,-0.003944,-1,1,0.0,0.0,66.0,7 days,False,True,False,False,True,False,False,1,0.01,0.72
2019-11-29,MSFT2019-12-06P150.00,2019-12-06,0.67,0.60,put,150.0,151.38,2019-11-29,7 days,-0.009116,-1,1,0.0,0.0,60.0,7 days,False,True,False,False,True,False,False,1,0.01,0.66
2019-12-06,MSFT2019-12-13P150.00,2019-12-13,0.60,0.49,put,150.0,151.75,2019-12-06,7 days,-0.011532,-1,0,0.0,0.0,49.0,7 days,False,True,False,False,True,False,False,1,0.01,0.59


In [32]:
short_close.shape

(142, 26)

In [33]:
short_close = short_close.sort_values('log', ascending = True)
short_close = short_close.dropna()
short_close = short_close.dropindex()
short_close_max_premium = short_close['log'].tail()

AttributeError: 'DataFrame' object has no attribute 'dropindex'

In [34]:
short_close_max_loss = short_close['log'].head()
short_close_max_loss

2019-12-11   -0.10
2019-12-11   -0.04
2019-12-06   -0.01
2019-11-21    0.00
2019-11-29    0.00
Name: log, dtype: float64

In [35]:
print(f'the biggest loses were', short_close_RF_total)

the biggest loses were 26.919999999999998


In [36]:
print('Selling options expiring in a week and close to the money,\n RF generated:', short_close_RF_total, '\n LTSM generated:', short_close_LTSM_total)
print('\nSelling options expiring in a week and slightly out to the money,\n RF generated:', short_medium_RF_total, '\n LTSM generated:', short_medium_LTSM_total)
print('\nSelling options expiring in a week and far out to the money,\n RF generated:', short_far_RF_total, '\n LTSM generated:', short_far_LTSM_total)


Selling options expiring in a week and close to the money,
 RF generated: 26.919999999999998 
 LTSM generated: 8.080000000000002

Selling options expiring in a week and slightly out to the money,
 RF generated: 0.010000000000000009 
 LTSM generated: -0.019999999999999907

Selling options expiring in a week and far out to the money,
 RF generated: 0.0 
 LTSM generated: 0.0


In [37]:
print('Selling options expiring in a two weeks and close to the money,\n RF generated:', long_close_RF_total, '\n LTSM generated:', long_close_LTSM_total)
print('\nSelling options expiring in two weeks and slightly out to the money,\n RF generated:', long_medium_RF_total, '\n LTSM generated:', long_medium_LTSM_total)
print('\nSelling options expiring in two weeks and far out to the money,\n RF generated:', long_far_RF_total, '\n LTSM generated:', long_far_LTSM_total)


Selling options expiring in a two weeks and close to the money,
 RF generated: 72.93 
 LTSM generated: 17.96

Selling options expiring in two weeks and slightly out to the money,
 RF generated: 0.4900000000000001 
 LTSM generated: 0.24000000000000007

Selling options expiring in two weeks and far out to the money,
 RF generated: 0.0 
 LTSM generated: 0.0
